# Actividad 5

Vamos a usar el dataset Fashion Mnist.

Los objetivos de esta actividad es predecir el objeto en la foto entrenado distinctas redes neuronales.

El notebook contiene basico codigo que descarga los datos y entrena y valida un modelo simple. Tu tienes que modificarlo.

In [1]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchsummary

import matplotlib.pyplot as plt

In [2]:
import os
import urllib.request
from sh import gunzip
import numpy as np
from sklearn.model_selection import train_test_split
from mlxtend.data import loadlocal_mnist

class FashionMnistLoader:
    
    dir_name = "data/fashion"
    url_train_imgs = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz"
    url_train_labels = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz"
    url_test_imgs = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz"
    url_test_labels = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz"
    
    def __init__(self):
        self.train_imgs_fn = None
        self.train_labels_fn = None
        self.test_imgs_fn = None
        self.test_labels_fn = None
        
    def get_all_data(self):
        self.train_imgs_fn = self.get_data(self.url_train_imgs)
        self.train_labels_fn = self.get_data(self.url_train_labels)
        self.test_imgs_fn = self.get_data(self.url_test_imgs)
        self.test_labels_fn = self.get_data(self.url_test_labels)
        return self
    
    def load_train(self):
        X, y = loadlocal_mnist(
            images_path=self.train_imgs_fn, 
            labels_path=self.train_labels_fn)
        return X, y
    
    def load_test(self):
        X, y = loadlocal_mnist(
            images_path=self.test_imgs_fn, 
            labels_path=self.test_labels_fn)
        return X, y
    
    def _split(self, X, y, test_size):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=666)
        return X_train, X_test, y_train, y_test
            
    def train_split(self, test_size):
        X, y = self.load_train()
        X_train, X_test, y_train, y_test = self._split(X, y, test_size)
        return X_train, X_test, y_train, y_test

    def standard_split(self):
        X_train, y_train = self.load_train()
        X_test, y_test = self.load_test()
        return X_train, X_test, y_train, y_test

    def get_data(self, url):
        gz_file_name = url.split("/")[-1]
        gz_file_path = os.path.join(self.dir_name, gz_file_name)  
        file_name = gz_file_name.split(".")[0]
        file_path = os.path.join(self.dir_name, file_name)
        os.makedirs(self.dir_name, exist_ok=True)
        if not os.path.exists(file_path):
            urllib.request.urlretrieve(url, gz_file_path) 
            gunzip(gz_file_path)  
        return file_path

In [3]:
data_loader = FashionMnistLoader().get_all_data()

X_train_dev, X_test, y_train_dev, y_test = data_loader.standard_split()
X_train_dev.shape, X_test.shape, len(y_train_dev), len(y_test)

((60000, 784), (10000, 784), 60000, 10000)

In [4]:
X_train, X_dev, y_train, y_dev = data_loader.train_split(1/6)
X_train.shape, X_dev.shape, len(y_train), len(y_dev)

((50000, 784), (10000, 784), 50000, 10000)

In [5]:
X_train_torch = torch.from_numpy(X_train).float()
X_dev_torch = torch.from_numpy(X_dev).float()
y_train_torch = torch.tensor(y_train.astype(np.int64))
y_dev_torch = torch.tensor(y_dev.astype(np.int64))

In [6]:
input_dim = 28 * 28

class LinearNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(input_dim, 10)
        
    def forward(self, x):
        x = self.fc(x)
        return x
    
simple_model = LinearNN()
torchsummary.summary(simple_model, input_size=((1, input_dim)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 10]           7,850
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


Esto es equivalente a lo siguente.

In [7]:
from collections import OrderedDict

simple_model2 = nn.Sequential(
    OrderedDict([
        ('fc', nn.Linear(in_features=input_dim, out_features=10, bias=True))
    ]))
torchsummary.summary(simple_model2, input_size=((1, input_dim)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 10]           7,850
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


In [8]:
learning_rate = 0.01

criterion = nn.CrossEntropyLoss()


In [9]:
from sklearn.metrics import accuracy_score
n_epoch = 1000

def train(model, n_epoch=n_epoch):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for i in range(n_epoch):
        # 0. Anular los gradientes.
        optimizer.zero_grad()
        # 1. Generar la salida
        output = model(X_train_torch)
        # 2. Calcular función de pérdida
        loss = criterion(output, y_train_torch)
        # 3. Backward propagation
        loss.backward()
        # 4. Cambiar los pesos
        optimizer.step()
    y_train_score = model(X_train_torch).detach().numpy()
    y_train_hat = np.argmax(y_train_score, axis=1)
    print(accuracy_score(y_train, y_train_hat))
    y_dev_score = model(X_dev_torch).detach().numpy()
    y_dev_hat = np.argmax(y_dev_score, axis=1)
    print(f"Accuracy: train {accuracy_score(y_train, y_train_hat)}, dev {accuracy_score(y_dev, y_dev_hat)}")

In [10]:
train(simple_model, 10)

0.58218
Accuracy: train 0.58218, dev 0.5831
